In [1]:
print('Hello world!')

Hello world!


# Python libs

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from cmath import rect, phase
from math import radians, degrees

# Remove warnings
import warnings
warnings.filterwarnings('ignore')

# Data import

In [3]:
# Set unlimited columns
pd.set_option('display.max_columns', None)

# Import CSV files to variables of type pandas.DataFrame
general_data = pd.read_csv('../csv/general_data.csv')
employee_survey_data = pd.read_csv('../csv/employee_survey_data.csv')
manager_survey_data = pd.read_csv('../csv/manager_survey_data.csv')
in_time = pd.read_csv('../csv/in_time.csv')
out_time = pd.read_csv('../csv/out_time.csv')

# Traitement de general_data

In [4]:
general_data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,1,Female,1,Healthcare Representative,Married,131160,1.0,Y,11,8,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,1,2,Female,1,Research Scientist,Single,41890,0.0,Y,23,8,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,1,3,Male,4,Sales Executive,Married,193280,1.0,Y,15,8,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,1,4,Male,3,Human Resources,Married,83210,3.0,Y,11,8,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,1,5,Male,1,Sales Executive,Single,23420,4.0,Y,12,8,2,9.0,2,6,0,4


On va vérifier la répartition des valeurs dans notre dataset general_data sur certaines colonnes. Si tous les employés ont la même valeur pour une même colonne alors cette colonne est inutile : autant l'éliminer.

Pour cela on utilisera la fonction value_counts : dans l'exemple ci-dessous on voit les différents âges et combien d'employés par valeur d'âge

In [5]:
 general_data["Age"].value_counts()

35    234
34    231
36    207
31    207
29    204
32    183
30    180
38    174
33    174
40    171
37    150
28    144
27    144
42    138
39    126
45    123
41    120
26    117
46     99
44     99
43     96
50     90
24     78
25     78
47     72
49     72
55     66
53     57
51     57
48     57
54     54
52     54
22     48
23     42
58     42
56     42
21     39
20     33
59     30
19     27
18     24
60     15
57     12
Name: Age, dtype: int64

Dans l'extrait du DataFrame plus haut, on repère seulement 3 colonnes qui ont la même valeur pour les 5 employés de l'extrait, on vérifie si cela se généralise dans tout le DataFrame ou si c'était un hasard avec les 5 premiers.

In [6]:
print(general_data["EmployeeCount"].value_counts())
print(general_data["Over18"].value_counts())
print(general_data["StandardHours"].value_counts())

1    4410
Name: EmployeeCount, dtype: int64
Y    4410
Name: Over18, dtype: int64
8    4410
Name: StandardHours, dtype: int64


Les colonnes EmployeeCount, Over18 et StandardHours sont les mêmes pour tous les employés, elles sont donc à éliminer.

In [7]:
general_data.drop(columns=['EmployeeCount', 'Over18','StandardHours'], inplace=True)
general_data.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EmployeeID,Gender,JobLevel,JobRole,MaritalStatus,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,1,Female,1,Healthcare Representative,Married,131160,1.0,11,0,1.0,6,1,0,0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,2,Female,1,Research Scientist,Single,41890,0.0,23,1,6.0,3,5,1,4
2,32,No,Travel_Frequently,Research & Development,17,4,Other,3,Male,4,Sales Executive,Married,193280,1.0,15,3,5.0,2,5,0,3
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,4,Male,3,Human Resources,Married,83210,3.0,11,3,13.0,5,8,7,5
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,5,Male,1,Sales Executive,Single,23420,4.0,12,2,9.0,2,6,0,4


On va transformer certaiens données présentes en  format numérique sous formes de caractères litteraux pour plus de compréhension. Ici Education peut être interessant à changer.

In [8]:
general_data["Education"] = general_data['Education'].replace({ 1 : 'Below College', 2: 'College',3: 'Bachelor',4: 'Master',5 : 'Doctor'})

On remarque que certaines colonnes : "NumCompaniesWorked" et "TotalWorkingYears" sont parfois non renseignées, on laisse les valeurs vides pour le moment mais il faudra peut-être les gérer si le besoin se présente.

# Traitement des questionnaires (employee_survey_data  et manager_survey_data)

In [9]:
employee_survey_data.head()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,1,3.0,4.0,2.0
1,2,3.0,2.0,4.0
2,3,2.0,2.0,1.0
3,4,4.0,4.0,3.0
4,5,4.0,1.0,3.0


 On va de la même façon que le premier fichier modifier nos colonnes numériques importantes en litteraux.

In [10]:
# employee_survey_data['WorkLifeBalance'] = employee_survey_data['WorkLifeBalance'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
# employee_survey_data['EnvironmentSatisfaction'] = employee_survey_data['EnvironmentSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
# employee_survey_data['JobSatisfaction'] = employee_survey_data['JobSatisfaction'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})

#employee_survey_data.head()

Pour le fichier des questionnaires managers, on fait pareil pour la note de l'implication et de la qualité de travail au cours de l'année

In [11]:
manager_survey_data.head()

,EmployeeID,JobInvolvement,PerformanceRating
0,1,3,3
1,2,2,4
2,3,3,3
3,4,2,3
4,5,3,3


In [12]:
#manager_survey_data['JobInvolvement'] = manager_survey_data['JobInvolvement'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})
#manager_survey_data['PerformanceRating'] = manager_survey_data['PerformanceRating'].replace({ 1 : 'Low', 2: 'Medium',3: 'High',4: 'Very High'})

#manager_survey_data.head()

Dans le questionnaires employés, ceux ci ne sont pas obligés de répondre, il faudra donc traiter les valeurs non renseignées pour notre apprentissage.

# Traitement des données sur les horaires de travail.

On va traiter deux fichiers : les horaires d'éntrées et de sorties pour chaque employé et pour chaque jour de l'anée.

In [13]:
# Reset in_time / out_time :
#in_time = pd.read_csv('../csv/in_time.csv')
#out_time = pd.read_csv('../csv/out_time.csv')

in_time.head()

,Unnamed: 0,2015-01-01,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-14,2015-01-15,2015-01-16,2015-01-19,2015-01-20,2015-01-21,2015-01-22,2015-01-23,2015-01-26,2015-01-27,2015-01-28,2015-01-29,2015-01-30,2015-02-02,2015-02-03,2015-02-04,2015-02-05,2015-02-06,2015-02-09,2015-02-10,2015-02-11,2015-02-12,2015-02-13,2015-02-16,2015-02-17,2015-02-18,2015-02-19,2015-02-20,2015-02-23,2015-02-24,2015-02-25,2015-02-26,2015-02-27,2015-03-02,2015-03-03,2015-03-04,2015-03-05,2015-03-06,2015-03-09,2015-03-10,2015-03-11,2015-03-12,2015-03-13,2015-03-16,2015-03-17,2015-03-18,2015-03-19,2015-03-20,2015-03-23,2015-03-24,2015-03-25,2015-03-26,2015-03-27,2015-03-30,2015-03-31,2015-04-01,2015-04-02,2015-04-03,2015-04-06,2015-04-07,2015-04-08,2015-04-09,2015-04-10,2015-04-13,2015-04-14,2015-04-15,2015-04-16,2015-04-17,2015-04-20,2015-04-21,2015-04-22,2015-04-23,2015-04-24,2015-04-27,2015-04-28,2015-04-29,2015-04-30,2015-05-01,2015-05-04,2015-05-05,2015-05-06,2015-05-07,2015-05-08,2015-05-11,2015-05-12,2015-05-13,2015-05-14,2015-05-15,2015-05-18,2015-05-19,2015-05-20,2015-05-21,2015-05-22,2015-05-25,2015-05-26,2015-05-27,2015-05-28,2015-05-29,2015-06-01,2015-06-02,2015-06-03,2015-06-04,2015-06-05,2015-06-08,2015-06-09,2015-06-10,2015-06-11,2015-06-12,2015-06-15,2015-06-16,2015-06-17,2015-06-18,2015-06-19,2015-06-22,2015-06-23,2015-06-24,2015-06-25,2015-06-26,2015-06-29,2015-06-30,2015-07-01,2015-07-02,2015-07-03,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-17,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-14,2015-09-15,2015-09-16,2015-09-17,2015-09-18,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-02,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-09,2015-11-10,2015-11-11,2015-11-12,2015-11-13,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-25,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,NaN,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,NaN,2015-01-15 10:01:24,2015-01-16 10:19:08,NaN,2015-01-20 09:50:34,2015-01-21 09:49:42,2015-01-22 09:47:45,2015-01-23 09:23:44,NaN,2015-01-27 09:50:37,2015-01-28 09:56:13,2015-01-29 09:53:47,2015-01-30 10:09:26,2015-02-02 09:38:43,2015-02-03 10:23:38,2015-02-04 09:48:37,2015-02-05 09:52:36,2015-02-06 09:53:23,2015-02-09 10:01:45,2015-02-10 10:14:18,2015-02-11 10:18:44,2015-02-12 10:10:35,2015-02-13 09:13:07,2015-02-16 10:14:02,2015-02-17 10:03:08,2015-02-18 10:23:06,NaN,2015-02-20 09:54:56,2015-02-23 09:46:59,2015-02-24 10:11:18,2015-02-25 09:52:36,2015-02-26 09:56:33,2015-02-27 09:58:46,2015-03-02 10:19:43,2015-03-03 10:08:37,2015-03-04 10:05:38,NaN,2015-03-06 09:55:53,2015-03-09 10:28:34,2015-03-10 09:47:30,2015-03-11 09:48:59,2015-03-12 10:10:51,2015-03-13 10:07:18,NaN,2015-03-17 10:05:22,2015-03-18 10:28:21,2015-03-19 10:01:52,2015-03-20 10:37:49,2015-03-23 10:11:19,2015-03-24 10:33:19,2015-03-25 09:41:35,NaN,2015-03-27 09:47:30,2

On va chercher les colonnes ou tous les employées n'ont pas été présents (NaN) : on supprimera celles ci.

In [14]:
holidays = in_time.columns[in_time.nunique() <= 1]

print(holidays)

Index(['2015-01-01', '2015-01-14', '2015-01-26', '2015-03-05', '2015-05-01',
       '2015-07-17', '2015-09-17', '2015-10-02', '2015-11-09', '2015-11-10',
       '2015-11-11', '2015-12-25'],
      dtype='object')


Ce dataframe répresente toute les dates pour lesquelles les employées n'étaient pas présents, on va les supprimer.

In [15]:
in_time = in_time.drop(columns = holidays)

in_time.head()

,Unnamed: 0,2015-01-02,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-12,2015-01-13,2015-01-15,2015-01-16,2015-01-19,2015-01-20,2015-01-21,2015-01-22,2015-01-23,2015-01-27,2015-01-28,2015-01-29,2015-01-30,2015-02-02,2015-02-03,2015-02-04,2015-02-05,2015-02-06,2015-02-09,2015-02-10,2015-02-11,2015-02-12,2015-02-13,2015-02-16,2015-02-17,2015-02-18,2015-02-19,2015-02-20,2015-02-23,2015-02-24,2015-02-25,2015-02-26,2015-02-27,2015-03-02,2015-03-03,2015-03-04,2015-03-06,2015-03-09,2015-03-10,2015-03-11,2015-03-12,2015-03-13,2015-03-16,2015-03-17,2015-03-18,2015-03-19,2015-03-20,2015-03-23,2015-03-24,2015-03-25,2015-03-26,2015-03-27,2015-03-30,2015-03-31,2015-04-01,2015-04-02,2015-04-03,2015-04-06,2015-04-07,2015-04-08,2015-04-09,2015-04-10,2015-04-13,2015-04-14,2015-04-15,2015-04-16,2015-04-17,2015-04-20,2015-04-21,2015-04-22,2015-04-23,2015-04-24,2015-04-27,2015-04-28,2015-04-29,2015-04-30,2015-05-04,2015-05-05,2015-05-06,2015-05-07,2015-05-08,2015-05-11,2015-05-12,2015-05-13,2015-05-14,2015-05-15,2015-05-18,2015-05-19,2015-05-20,2015-05-21,2015-05-22,2015-05-25,2015-05-26,2015-05-27,2015-05-28,2015-05-29,2015-06-01,2015-06-02,2015-06-03,2015-06-04,2015-06-05,2015-06-08,2015-06-09,2015-06-10,2015-06-11,2015-06-12,2015-06-15,2015-06-16,2015-06-17,2015-06-18,2015-06-19,2015-06-22,2015-06-23,2015-06-24,2015-06-25,2015-06-26,2015-06-29,2015-06-30,2015-07-01,2015-07-02,2015-07-03,2015-07-06,2015-07-07,2015-07-08,2015-07-09,2015-07-10,2015-07-13,2015-07-14,2015-07-15,2015-07-16,2015-07-20,2015-07-21,2015-07-22,2015-07-23,2015-07-24,2015-07-27,2015-07-28,2015-07-29,2015-07-30,2015-07-31,2015-08-03,2015-08-04,2015-08-05,2015-08-06,2015-08-07,2015-08-10,2015-08-11,2015-08-12,2015-08-13,2015-08-14,2015-08-17,2015-08-18,2015-08-19,2015-08-20,2015-08-21,2015-08-24,2015-08-25,2015-08-26,2015-08-27,2015-08-28,2015-08-31,2015-09-01,2015-09-02,2015-09-03,2015-09-04,2015-09-07,2015-09-08,2015-09-09,2015-09-10,2015-09-11,2015-09-14,2015-09-15,2015-09-16,2015-09-18,2015-09-21,2015-09-22,2015-09-23,2015-09-24,2015-09-25,2015-09-28,2015-09-29,2015-09-30,2015-10-01,2015-10-05,2015-10-06,2015-10-07,2015-10-08,2015-10-09,2015-10-12,2015-10-13,2015-10-14,2015-10-15,2015-10-16,2015-10-19,2015-10-20,2015-10-21,2015-10-22,2015-10-23,2015-10-26,2015-10-27,2015-10-28,2015-10-29,2015-10-30,2015-11-02,2015-11-03,2015-11-04,2015-11-05,2015-11-06,2015-11-12,2015-11-13,2015-11-16,2015-11-17,2015-11-18,2015-11-19,2015-11-20,2015-11-23,2015-11-24,2015-11-25,2015-11-26,2015-11-27,2015-11-30,2015-12-01,2015-12-02,2015-12-03,2015-12-04,2015-12-07,2015-12-08,2015-12-09,2015-12-10,2015-12-11,2015-12-14,2015-12-15,2015-12-16,2015-12-17,2015-12-18,2015-12-21,2015-12-22,2015-12-23,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
0,1,2015-01-02 09:43:45,2015-01-05 10:08:48,2015-01-06 09:54:26,2015-01-07 09:34:31,2015-01-08 09:51:09,2015-01-09 10:09:25,2015-01-12 09:42:53,2015-01-13 10:13:06,2015-01-15 10:01:24,2015-01-16 10:19:08,NaN,2015-01-20 09:50:34,2015-01-21 09:49:42,2015-01-22 09:47:45,2015-01-23 09:23:44,2015-01-27 09:50:37,2015-01-28 09:56:13,2015-01-29 09:53:47,2015-01-30 10:09:26,2015-02-02 09:38:43,2015-02-03 10:23:38,2015-02-04 09:48:37,2015-02-05 09:52:36,2015-02-06 09:53:23,2015-02-09 10:01:45,2015-02-10 10:14:18,2015-02-11 10:18:44,2015-02-12 10:10:35,2015-02-13 09:13:07,2015-02-16 10:14:02,2015-02-17 10:03:08,2015-02-18 10:23:06,NaN,2015-02-20 09:54:56,2015-02-23 09:46:59,2015-02-24 10:11:18,2015-02-25 09:52:36,2015-02-26 09:56:33,2015-02-27 09:58:46,2015-03-02 10:19:43,2015-03-03 10:08:37,2015-03-04 10:05:38,2015-03-06 09:55:53,2015-03-09 10:28:34,2015-03-10 09:47:30,2015-03-11 09:48:59,2015-03-12 10:10:51,2015-03-13 10:07:18,NaN,2015-03-17 10:05:22,2015-03-18 10:28:21,2015-03-19 10:01:52,2015-03-20 10:37:49,2015-03-23 10:11:19,2015-03-24 10:33:19,2015-03-25 09:41:35,NaN,2015-03-27 09:47:30,2015-03-30 10:11:44,2015-03-31 10:08:13,2015-04-01 10:12:37,2015-04-02 09:45:27,2015-04-03 10:00:30,2015-04-06 09:40:19,2015-04-07 10:16:43,2015-04-0

Maintenant on va remplacer les valeurs non définies par 0 et convertir l'ensemble au format datetime

In [ ]:
in_time = in_time.replace(np.nan, 0)
in_time.iloc[:, 1:] = in_time.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

in_time.head()

On fait les mêmes manipulations avec le dataframe des horraires de départ

In [ ]:
out_time = out_time.drop(columns = holidays)

out_time = out_time.replace(np.nan,0)
out_time.iloc[:, 1:] = out_time.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')

out_time.head()

Pour notre phase d'apprentissage, il est important pour nous de mettre en place des moyennes.

On va commencer par faire une moyenne du temps de travail pour chaque employé.

Pour mieux y voir, on va mélanger nos deux dataframes précedent : 

In [ ]:
in_out_time = in_time.append(out_time)

in_out_time.shape

In [ ]:
employees_number = (int) (in_out_time.shape[0] / 2)

# On fait la différence entre l'heure de départ et l'heure d'arrivée
in_out_time = in_out_time.diff(periods=employees_number)
# La seconde moitié du tableau (anciennement out_time) est désormais aux valeurs de durée

# On supprime la première moitié du tableau (in_time) puis on reset les index
in_out_time = in_out_time.iloc[employees_number:]
in_out_time.reset_index(inplace=True)

in_out_time.head()

Nos deux premières colonnes sont inutiles, on va les supprimer

In [ ]:
in_out_time.drop(columns=['index', 'Unnamed: 0'], axis=1, inplace=True)
in_out_time.head()

On crée une colonne contenant la moyenne de ces temps de travail pour chaque employé

In [ ]:
in_out_time['AverageWorkTimeDelta'] = in_out_time.mean(axis=1)

in_out_time.head()

On peut transformer la moyenne d'heure en heures pour plus de clarté

In [ ]:
in_out_time['AverageWorkTime'] = in_out_time['AverageWorkTimeDelta'] / np.timedelta64(1, 'h')
in_out_time.head()

On va maintenant faire une moyenne des heures d'arrivée et de départ de chaque employé.

In [ ]:
def mean_angle(deg):
    return degrees(phase(sum(rect(1, radians(d)) for d in deg)/len(deg)))
 
def mean_time(row):
    t = ([time.hour, time.minute, time.second] for time in row)
    seconds = ((float(s) + int(m) * 60 + int(h) * 3600) 
               for h, m, s in t)
    day = 24 * 60 * 60
    to_angles = [s * 360. / day for s in seconds]
    mean_as_angle = mean_angle(to_angles)
    mean_seconds = mean_as_angle * day / 360.
    if mean_seconds < 0:
        mean_seconds += day
    h, m = divmod(mean_seconds, 3600)
    m, s = divmod(m, 60)
    return '%02i:%02i:%02i' % (h, m, s)

in_out_time['AverageInTime'] = in_time.drop(['Unnamed: 0'], 1).apply(mean_time, axis=1)
in_out_time['AverageOutTime'] = out_time.drop(['Unnamed: 0'], 1).apply(mean_time, axis=1)

On va désormais drop les colonnes de chaque jour qui ne nous sont plus utiles

In [ ]:
# EmployeeID commence à 1 au lieu de 0
in_out_time.index += 1

in_out_time.reset_index(inplace=True)
in_out_time.rename(columns={'index': 'EmployeeID'}, inplace=True)
in_out_time.drop(in_out_time.columns.difference(['EmployeeID', 'AverageWorkTime', 'NumberOfAbsences', 'AverageInTime', 'AverageOutTime']), 1, inplace=True)

in_out_time.head()

# Fusion en un seul DataFrame
On s'occupe maintenant de fusionner toutes les données en un seul DataFrame, en faisant correspondre les EmployeeID.
Ce DataFrame constituera notre le jeu de donnée de base du projet à partir d'ici.

In [ ]:
df = pd.merge(employee_survey_data, manager_survey_data, how='inner', on='EmployeeID')
df = pd.merge(df, general_data, how='inner', on='EmployeeID')
df = pd.merge(df, in_out_time, how='inner', on='EmployeeID')

df.head()

# ---- Essais statistiques ---

In [ ]:
df.describe()

In [ ]:
df.info()

## Mise en évidence de la variable clé Attrition

In [ ]:
print(round(df['Attrition'].value_counts(normalize = True),2))
sns.countplot(x='Attrition',data=df)

In [ ]:
sns.pairplot(df[['Age','MonthlyIncome','DistanceFromHome','Attrition']],hue = 'Attrition')

In [ ]:
print('Fréquence de voyages (en %): \n')
print(round(df['BusinessTravel'].value_counts(normalize = True)*100,2))
print("\n Taux d'attrition par type de voyages \n")
print(round(df['BusinessTravel'][df['Attrition'] == 'Yes'].value_counts()/df['BusinessTravel'].value_counts()*100,2))

In [ ]:
print("Nombres d'émployés par département (en %): \n")
print(round(df['Department'].value_counts(normalize = True)*100,2))  
print("\n Taux d'attrition par département \n")
print(round(df['Department'][df['Attrition'] == 'Yes'].value_counts()/df['Department'].value_counts()*100,2))

In [ ]:
print("Nombre d'émployées par niveau d'éducation (en %): \n")
print(round(df['Education'].value_counts(normalize = True)*100,2).sort_index())
print("\n Taux d'attrition par niveau d'éducation : \n")
print(round(df['Education'][df['Attrition'] == 'Yes'].value_counts()/df['Education'].value_counts()*100,2).sort_index())

On peut chercher d'autre types de corrélation comme celle entre l'attarition et le statut marital.

In [ ]:
print("\n Taux d'attrition par statut marital: \n")
print(round(df['MaritalStatus'][df['Attrition'] == 'Yes'].value_counts()/df['MaritalStatus'].value_counts()*100,2))
(df['MaritalStatus'][df['Attrition'] == 'Yes'].value_counts()/df['MaritalStatus'].value_counts()*100).plot.bar(color = 'blue')

On peut également chercher le nombre d'heures moyen et le taux d'attrition

In [ ]:
sns.pairplot(df[['AverageWorkTime', 'JobSatisfaction', 'Attrition']],hue = 'Attrition', height = 4)

## Traitement des données 

Certaines de nos colonnes ne sont pas entièrement remplies : on va les remplir avec les valeurs médianes adéquates.

In [ ]:
print(df.isnull().sum())

### EnvironmentSatisfaction

In [ ]:
sns.countplot(x='EnvironmentSatisfaction', data=df)

In [ ]:
df['EnvironmentSatisfaction'].value_counts(ascending=False)

In [ ]:
df["EnvironmentSatisfaction"].fillna(df["EnvironmentSatisfaction"].median(), inplace=True)


### JobSatisfaction

In [ ]:
sns.countplot(x='JobSatisfaction', data=df)

In [ ]:
df['JobSatisfaction'].value_counts(ascending=False)

In [ ]:
df["JobSatisfaction"].fillna(df["JobSatisfaction"].median(), inplace=True)
df['JobSatisfaction'].isnull().sum()

## WorkLifeBalance

In [ ]:
df['WorkLifeBalance'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='WorkLifeBalance',data=df);

In [ ]:
df["WorkLifeBalance"].fillna(df["WorkLifeBalance"].median(), inplace=True)
df['WorkLifeBalance'].isnull().sum()

## NumCompaniesWorked

In [ ]:
df['NumCompaniesWorked'].value_counts(ascending=False)

In [ ]:
sns.countplot(x='NumCompaniesWorked',data=df);

In [ ]:
sns.boxplot(x='NumCompaniesWorked',data=df);

In [ ]:
df['NumCompaniesWorked'].fillna(df.groupby(['TotalWorkingYears'])['NumCompaniesWorked'].transform('median'), inplace=True)
df['NumCompaniesWorked'].isnull().sum()

## TotalWorkingYears

In [ ]:
df['TotalWorkingYears'].value_counts(ascending=False)

In [ ]:
plt.figure(figsize=(8,8))
ax = sns.distplot(df['TotalWorkingYears'], hist=True, kde=False, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
ax.set_ylabel('# of Employees')
ax.set_xlabel('TotalWorkingYears');

#Expliquer le schéma. kde = gaussian kernel density estimate => courbe.

In [ ]:
sns.boxplot(x='TotalWorkingYears',data=df);


In [ ]:
df['TotalWorkingYears'].fillna(df.groupby(['Age'])['TotalWorkingYears'].transform('median'), inplace=True)
df['TotalWorkingYears'].isnull().sum()

In [ ]:
df['TotalWorkingYears'].head()

In [ ]:
df.info()


On à désormais une harmonisation de donéés grâce à notre traitement de valeurs nulles ou non renseignées.